# Build NN models

Creates a sequence of NN models to predict the increment of the inverse CDF with each percentile.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import copy

import pandas as pd
import seaborn as sns
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm
#from tensorflow.keras.layers.merge import Maximum, Minimum

#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

In [2]:
def neurons(qi):
    #return int(2**(4 + 0.01*qi*3))

    return int(16 + (128 - 16)*(qi-1)/float(98))

In [3]:
def shift_cdf(a):
    for i in range(a.shape[1]-1,3,-1):
        a[:,i] = a[:,i] - a[:,i-1]
        
    return a

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
data_directory = './training_data'

os.chdir(data_directory)

cdf_training_files = glob('*survey.csv')

In [6]:
total_training_set = {}

c = np.linspace(1,99,99,dtype=int)
column_names = [str(ic) for ic in c]

gs = np.linspace(1,11,11,dtype=int)
g_names = ['g'+str(i) for i in gs]

#column_names = ['c']+g_names+['g']+column_names
#column_names = ['c']+['g']+column_names
column_names = ['g']+column_names
#print(column_names)

for i in range(0,len(cdf_training_files)):
    if i==0:
        a = pd.read_csv(cdf_training_files[i],header=None).to_numpy()
        
        concs = np.zeros(shape=(a.shape[0],1))
        concs[:,0] = a[:,0]
        
        response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

        for j in range(0,a.shape[0]):
            response_labels[j,:] = a[:,1]
         
        for j in range(0,a.shape[0]):
            if a[j,2]==0.0:
                a[j,2] = 0.5*a[j,3]
        #print(concs.shape,response_labels.shape,a[:,1:].shape)
            
        #data_array = np.zeros(shape=(a.shape[0],(a.shape[1]+response_labels.shape[1])))
        
        #aa = shift_cdf(a)
        
        #data_array = np.concatenate((concs,aa[:,1:]),axis=1)
        mean_array = np.zeros(shape=(a[:,1].shape[0],1))
        mean_array[:,0] = a[:,1]
        #data_array = np.concatenate((concs,mean_array,a[:,3:-1]),axis=1)
        
        data_array = np.concatenate((mean_array,a[:,3:-1]),axis=1)
        
        #print(data_array)
    else:
        a = pd.read_csv(cdf_training_files[i],header=None).to_numpy()
        
        concs = np.zeros(shape=(a.shape[0],1))
        concs[:,0] = a[:,0]
        
        response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

        for j in range(0,a.shape[0]):
            response_labels[j,:] = a[:,1]
        
        for j in range(0,a.shape[0]):
            if a[j,2]==0.0:
                a[j,2] = 0.5*a[j,3]
            
        #aa = shift_cdf(a)
            
        #this_array = np.concatenate((concs,aa[:,1:]),axis=1)
        mean_array = np.zeros(shape=(a[:,1].shape[0],1))
        mean_array[:,0] = a[:,1]
        #this_array = np.concatenate((concs,mean_array,a[:,3:-1]),axis=1)
        
        this_array = np.concatenate((mean_array,a[:,3:-1]),axis=1)
        
        #print(i,cdf_training_files[i],this_array.shape,data_array.shape)
        #sys.stdout.flush()
            
        data_array = np.concatenate((data_array,this_array),axis=0)
        
total_training_set = pd.DataFrame(data_array,index=[i for i in range(data_array.shape[0])],columns=column_names)
#total_training_set[i] = pd.read_csv(cdf_training_files[i],names=column_names)
#print(total_training_set['g'])
print(data_array.shape)
print(list(total_training_set))

(1200, 100)
['g', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [7]:
training_labels = {}
training_sets = {}

keyset = list(np.linspace(1,data_array.shape[1]-1,99,dtype=int))
keyset.reverse()
print(keyset)

for i in keyset:
    #print(i,column_names[i])
    #training_labels[i-13] = total_training_set.pop(column_names[i])
    #training_sets[i-13] = copy.deepcopy(total_training_set)
    #print(column_names[i])
    training_labels[i-1] = total_training_set.pop(column_names[i])
    training_sets[i-1] = copy.deepcopy(total_training_set)
    
    if i>=3:   
        #if i==3:
        #    print(training_labels[i-2])
            
        training_labels[i-1] = training_labels[i-1] - total_training_set[str(i-2)]
        
        #if i==3:
        #    print(training_labels[i-2])
        #    print(total_training_set[str(i-3)])
keyset.reverse()

keyset = np.array(keyset,dtype=int) - 2*np.ones(shape=(len(keyset),),dtype=int)

[99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


**Train and save models**

In [ ]:
data_directory = '../models/'
os.chdir(data_directory)

model_name = 'linearly-growing-layers-2'

try:
    os.chdir(model_name)
except OSError:
    os.mkdir(model_name)
    os.chdir(model_name)

EPOCHS = 500

training_loss = np.zeros(shape=(99,3))
validation_loss = np.zeros(shape=(99,3))

for i in tqdm(range(0,99)):
    #plt.close()
    def build_model(qi):
        if qi>=0:
            n_neurons = neurons(qi)
            
            print('Neurons: ',n_neurons)
            
            model = keras.Sequential([
            layers.Dense(n_neurons, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(n_neurons, activation='relu'),
            layers.Dense(1)
            ])
        """
        elif qi>=50:
            model = keras.Sequential([
            layers.Dense(32, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(32, activation='relu'),
            #layers.Dense(64, activation='relu'),
            layers.Dense(1)
            ])
        elif qi>=25:
            model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1)
            ])
        else:
            model = keras.Sequential([
            layers.Dense(128, activation='relu', input_shape=[len(training_sets[i].keys())]),
            layers.Dense(128, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(1)
            ])
        """

        optimizer = tf.keras.optimizers.RMSprop(0.001,0.9,centered=False)

        #model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])
        
        model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])
        
        return model
    
    model = build_model(i)

    #model.summary()
    
    j = i - 1
    
    #history = model.fit(training_sets[i], training_labels[i],
    #  epochs=EPOCHS, validation_split = 0.3, verbose=0, batch_size=12, shuffle=True)#callbacks=[tfdocs.modeling.EpochDots()])
    
    recasted_input = tf.expand_dims(training_sets[i], axis=-1)
    
    print(recasted_input.shape,training_labels[i].shape)
    
    history = model.fit(recasted_input, training_labels[i], epochs=EPOCHS, validation_split = 0.3, verbose=0, batch_size=12, shuffle=True)#callbacks=[tfdocs.modeling.EpochDots()])
    
    loss_array = np.zeros(shape=(len(history.history['loss']),3))
    loss_array[:,0] = np.linspace(1,loss_array.shape[0],loss_array.shape[0])
    loss_array[:,1] = np.array(history.history['loss'])
    loss_array[:,2] = np.array(history.history['val_loss'])
    
    np.savetxt('loss_array'+str(i)+'.csv',loss_array,delimiter=',')
        
    training_loss[j,0] = history.history['loss'][0]
    training_loss[j,1] = history.history['loss'][-1]
    training_loss[j,2] = history.history['loss'][-1]/history.history['loss'][0]
    
    validation_loss[j,0] = history.history['val_loss'][0]
    validation_loss[j,1] = history.history['val_loss'][-1]
    validation_loss[j,2] = history.history['val_loss'][-1]/history.history['val_loss'][0]
    
    print('Loss(training, validation): ',training_loss[j,2],validation_loss[j,2])
     
    model_name = 'model_'+str(i)+'.h5'
    model.save(model_name) 

  0%|          | 0/99 [00:00<?, ?it/s]

Neurons:  14
(1200, 1, 1) (1200,)
Loss(training, validation):  0.08491944047300827 0.23968204037701404
Neurons:  16
(1200, 2, 1) (1200,)
Loss(training, validation):  0.2264726618138984 2.196529066500324
Neurons:  17
(1200, 3, 1) (1200,)
Loss(training, validation):  0.11757672790763551 0.06792831583477614
Neurons:  18
(1200, 4, 1) (1200,)
Loss(training, validation):  0.03002082957154281 0.047760847522767866
Neurons:  19
(1200, 5, 1) (1200,)
Loss(training, validation):  0.02060230810888833 1.4640174495354736
Neurons:  20
(1200, 6, 1) (1200,)
Loss(training, validation):  0.018289077452782205 0.13449782234366867
Neurons:  21
(1200, 7, 1) (1200,)
Loss(training, validation):  0.0017875199230545766 0.5768888416278062
Neurons:  22
(1200, 8, 1) (1200,)
Loss(training, validation):  0.008102650508279751 0.44576865705317487
Neurons:  24
(1200, 9, 1) (1200,)
Loss(training, validation):  0.0011094274770933983 0.07406849969034784
Neurons:  25
(1200, 10, 1) (1200,)
Loss(training, validation):  6.67688

In [ ]:
np.savetxt('validation_loss.csv',validation_loss,delimiter=',')
np.savetxt('training_loss.csv',training_loss,delimiter=',')